# Variables initialization


In [ ]:
# Aks compute name
akscomputename = 'akscntademo'

# Connect to an Azure Machine Learning Workspace

The Azure Machine Learning Python SDK is required for leveraging the experimentation, model management and model deployment capabilities of Azure Machine Learning services. Run the following cell to create a new Azure Machine Learning **Workspace** and save the configuration to disk. The configuration file named `config.json` is saved in a folder named `.azureml`. 

**Important Note**: You might be prompted to login in the text that is output below the cell. Be sure to navigate to the URL displayed and enter the code that is provided. Once you have entered the code, return to this notebook and wait for the output to read `Workspace configuration succeeded`.

In [ ]:
import azureml.core
print('azureml.core.VERSION: ', azureml.core.VERSION)

# import the Workspace class and check the azureml SDK version
from azureml.core import Workspace
ws = Workspace.from_config()
print(ws)
print('Workspace configuration succeeded')

# Deploy model to Azure Kubernetes Services (AKS)

In this section, you will deploy a web service that uses Gensim as shown in `01 Summarize` to summarize text. The web service will be hosted in Azure Kubernetes Services.

## Create the scoring web service

When deploying models for scoring with Azure Machine Learning services, you need to define the code for a simple web service that will load your model and use it for scoring. By convention this service has two methods init which loads the model and run which scores data using the loaded model.

This scoring service code will later be deployed inside of a specially prepared Docker container.

In [ ]:
%%writefile summarizer_service.py

import re
import nltk
import unicodedata
from gensim.summarization import summarize, keywords

def clean_and_parse_document(document):
    if isinstance(document, str):
        document = document
    elif isinstance(document, unicode):
        return unicodedata.normalize('NFKD', document).encode('ascii', 'ignore')
    else:
        raise ValueError("Document is not string or unicode.")
    document = document.strip()
    sentences = nltk.sent_tokenize(document)
    sentences = [sentence.strip() for sentence in sentences]
    return sentences

def summarize_text(text, summary_ratio=None, word_count=30):
    sentences = clean_and_parse_document(text)
    cleaned_text = ' '.join(sentences)
    summary = summarize(cleaned_text, split=True, ratio=summary_ratio, word_count=word_count)
    return summary 

def init():  
    nltk.download('punkt')
    return

def run(input_str):
    try:
        return summarize_text(input_str)
    except Exception as e:
        return (str(e))

## Environments

Azure ML environments are an encapsulation of the environment where your machine learning training happens. They define Python packages, environment variables, Docker settings and other attributes in declarative fashion. Environments are versioned: you can update them and retrieve old versions to revisit and review your work.

Environments allow you to:
* Encapsulate dependencies of your training process, such as Python packages and their versions.
* Reproduce the Python environment on your local computer in a remote run on VM or ML Compute cluster
* Reproduce your experimentation environment in production setting.
* Revisit and audit the environment in which an existing model was trained.

Environment, compute target and training script together form run configuration: the full specification of training run.

### Use curated environments

Curated environments are provided by Azure Machine Learning and are available in your workspace by default. They contain collections of Python packages and settings to help you get started different machine learning frameworks. 

  * The __AzureML-Minimal__ environment contains a minimal set of packages to enable run tracking and asset uploading. You can use it as a starting point for your own environment.
  * The __AzureML-Tutorial__ environment contains common data science packages, such as Scikit-Learn, Pandas and Matplotlib, and larger set of azureml-sdk packages.
 
Curated environments are backed by cached Docker images, reducing the run preparation cost.

See https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/training/using-environments/using-environments.ipynb for more details.

### Create your own environment

Instead of curated environments, may create an environment by instantiating ```Environment``` object and then setting its attributes: set of Python packages, environment variables and others.  We will take this approach in this training.

#### Add Python packages

The recommended way is to specify Conda packages, as they typically come with complete set of pre-built binaries. Still, you may also add pip packages, and specify the version of package.

In [ ]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

myenv = Environment(name="myenv")
conda_dep = CondaDependencies()
conda_dep.add_pip_package("gensim==3.8.3")
conda_dep.add_pip_package("nltk==3.4.5")
myenv.python.conda_dependencies=conda_dep

### Register environment

You can manage environments by registering them. This allows you to track their versions, and reuse them in future runs. For example, once you've constructed an environment that meets your requirements, you can register it and use it in other experiments so as to standardize your workflow.

If you register the environment with same name, the version number is increased by one. Note that Azure ML keeps track of differences between the version, so if you re-register an identical version, the version number is not increased.

In [ ]:
myenv.register(workspace=ws)

## Deployment

If you want more control over how your model is run, if it uses another framework, or if it has special runtime requirements, you can instead specify your own environment and scoring method. Custom environments can be used for any model you want to deploy.

In previous code, you specified the model's runtime environment by creating an [Environment](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment%28class%29?view=azure-ml-py) object and providing the [CondaDependencies](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.conda_dependencies.condadependencies?view=azure-ml-py) needed by your model.

In the following cells you will use the Azure Machine Learning SDK to package the model and scoring script in a container, and deploy that container to an Azure Kubernetes cluster.

Run the following cells.

In [ ]:
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.model import Model
from azureml.core.compute import AksCompute
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core import Model
from azureml.core import Webservice
from azureml.exceptions import WebserviceException


inference_config = InferenceConfig(entry_script='summarizer_service.py', environment=myenv)

aks_target = AksCompute(ws,akscomputename)
service_name = "summarizer"
deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service = Model.deploy(ws, service_name,[], inference_config, deployment_config, aks_target)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())

Now you are ready to begin your deployment to the AKS.  In this case, the model is inside the Python script, so the _model_ parameter has a blank list.

Run the following cell:  you may be waiting 5-15 minutes for completion, while the _Running_ tag adds progress dots.

You will see output similar to the following when your web service is ready: 

`
Succeeded
AKS service creation operation finished, operation "Succeeded"`

## Test the deployed service

Now you are ready to test scoring using the deployed web service. The following cell invokes the web service.

Run the following cells to test scoring using a single input row against the deployed web service.

**Load the deployed webservice from workspace**

In [ ]:
from azureml.core.webservice import Webservice
service_name = "summarizer"
webservice = Webservice(ws, service_name)
webservice

In [ ]:
example_document = """
I was driving down El Camino and stopped at a red light.
It was about 3pm in the afternoon.  
The sun was bright and shining just behind the stoplight.
This made it hard to see the lights.
There was a car on my left in the left turn lane.
A few moments later another car, a black sedan pulled up behind me. 
When the left turn light changed green, the black sedan hit me thinking 
that the light had changed for us, but I had not moved because the light 
was still red.
After hitting my car, the black sedan backed up and then sped past me.
I did manage to catch its license plate. 
The license plate of the black sedan was ABC123. 
"""

In [ ]:
result = webservice.run(input_data = example_document)
print(result)

## Capture the scoring URI

In order to call the service from a REST client, you need to acquire the scoring URI. Take a note of printed scoring URI, you will need it in the last notebook.

The default settings used in deploying this service result in a service that does not require authentication, so the scoring URI is the only value you need to call this service.

In [ ]:
url = webservice.scoring_uri
print('AKS Service: Summarizer scoring URI is: {}'.format(url))